In [1]:
import os

class Reader(object):
    def __init__(self, xml_dir):
        self.xml_dir = xml_dir

    def get_xml_files(self):
        xml_filenames = []
        for root, subdirectories, files in os.walk(self.xml_dir):
            for filename in files:
                if filename.endswith(".xml"):
                    file_path = os.path.join(root, filename)
                    file_path = os.path.relpath(file_path, start=self.xml_dir)
                    xml_filenames.append(file_path)    
        return xml_filenames

    @staticmethod
    def get_classes(filename):
        with open(os.path.join(os.path.dirname(os.path.realpath('__file__')), filename), "r", encoding="utf8") as f:
            lines = f.readlines()
            return {value: key for (key, value) in enumerate(list(map(lambda x: x.strip(), lines)))}

In [2]:
import logging
import os
import declxml as xml

class ObjectMapper(object):
    def __init__(self):
        self.processor = xml.user_object("annotation", Annotation, [
            xml.user_object("size", Size, [
                xml.integer("width"),
                xml.integer("height"),
            ]),
            xml.array(
                xml.user_object("object", Object, [
                    xml.string("name"),
                    xml.user_object("bndbox", Box, [
                        xml.integer("xmin"),
                        xml.integer("ymin"),
                        xml.integer("xmax"),
                        xml.integer("ymax"),
                    ], alias="box")
                ]),
                alias="objects"
            ),
            xml.string("filename")
        ])

    def bind(self, xml_file_path, xml_dir):
        ann = xml.parse_from_file(self.processor, xml_file_path=os.path.join(xml_dir, xml_file_path))
        ann.filename = xml_file_path
        return ann

    def bind_files(self, xml_file_paths, xml_dir):
        result = []
        for xml_file_path in xml_file_paths:
            try:
                result.append(self.bind(xml_file_path=xml_file_path, xml_dir=xml_dir))
            except Exception as e:
                logging.error("%s", e.args)
        return result


class Annotation(object):
    def __init__(self):
        self.size = None
        self.objects = None
        self.filename = None

    def __repr__(self):
        return "Annotation(size={}, object={}, filename={})".format(self.size, self.objects, self.filename)


class Size(object):
    def __init__(self):
        self.width = None
        self.height = None

    def __repr__(self):
        return "Size(width={}, height={})".format(self.width, self.height)


class Object(object):
    def __init__(self):
        self.name = None
        self.box = None

    def __repr__(self):
        return "Object(name={}, box={})".format(self.name, self.box)


class Box(object):
    def __init__(self):
        self.xmin = None
        self.ymin = None
        self.xmax = None
        self.ymax = None

    def __repr__(self):
        return "Box(xmin={}, ymin={}, xmax={}, ymax={})".format(self.xmin, self.ymin, self.xmax, self.ymax)

In [3]:
xml_dir = os.path.join(os.path.dirname(os.path.realpath('__file__')), 'xml')
reader = Reader(xml_dir=xml_dir)

xml_files = reader.get_xml_files()
print(xml_files[0])
print(len(xml_files))

class_file = os.path.join(os.path.dirname(os.path.realpath('__file__')), 'classes.txt')
classes = reader.get_classes(class_file)
print(classes)

object_mapper = ObjectMapper()
annotations = object_mapper.bind_files(xml_files, xml_dir=xml_dir)
print(annotations[0])
print(type(annotations[0]))
print(annotations[0].size.width)
print(annotations[0].filename.split('.')[0] + '.png')

D_0.xml
715
{'Traffic Signal': 0, 'Lamp Post': 1, 'Zebra Crossing': 2, 'Bike': 3, 'Car': 4, 'Rikshaw': 5, 'Tyre Works': 6, 'Tree': 7, 'Tractor': 8, 'Cattle': 9, 'Vegetation': 10, 'Electricity Pole': 11, 'Building': 12, 'Board': 13, 'Wall': 14, 'person': 15, 'Bus': 16, 'Bridge': 17, 'Road Divider': 18, 'Tempo': 19, 'Traffic Sign Board': 20, 'Flag': 21, 'Crane': 22, 'Cycle': 23, 'Dog': 24, 'Truck': 25, 'WWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWW': 26, 'Overbridge': 27, 'Manhole': 28, 'Bus Stop': 29, 'Barricade': 30, 'Petrol Pump': 31, 'Ambulance': 32, 'Goat': 33, 'Cart': 34}
Annotation(size=Size(width=0, height=0), object=[Object(name=Lamp Post, box=Box(xmin=19, ymin=456, xmax=641, ymax=1656)), Object(name=Tree, box=Box(xmin=1305, ymin=683, xmax=1669, ymax=1674)), Object(name=Bike, box=Box(xmin=123, ymin=1897, xmax=341, ymax=2147)), Object(name=Rikshaw, box=Box(xmin=691, ymin=1674, xmax=1059, ymax=2179)), Object(name=Car, box=Box(xmin=1287, ymin=173

In [4]:
import os
import sys
from PIL import Image

class Transformer(object):
    def __init__(self, xml_dir, out_dir, class_file, img_dir):
        self.xml_dir = xml_dir
        self.out_dir = out_dir
        self.class_file = class_file
        self.img_dir = img_dir

    def transform(self):
        reader = Reader(xml_dir=self.xml_dir)
        xml_files = reader.get_xml_files()
        classes = reader.get_classes(self.class_file)
        object_mapper = ObjectMapper()
        annotations = object_mapper.bind_files(xml_files, xml_dir=self.xml_dir)
        self.write_to_txt(annotations, classes)

    def write_to_txt(self, annotations, classes):
        for annotation in annotations:
            output_path = os.path.join(self.out_dir, self.darknet_filename_format(annotation.filename))
            img_path = os.path.join(self.img_dir, annotation.filename.split('.')[0] + '.png')

            try:
                img = Image.open(img_path)
            except FileNotFoundError:
                continue

            annotation.size.height = img.height
            annotation.size.width = img.width

            if not os.path.exists(os.path.dirname(output_path)):
                os.makedirs(os.path.dirname(output_path))
            with open(output_path, "w+") as f:
                f.write(self.to_darknet_format(annotation, classes))

    def to_darknet_format(self, annotation, classes):
        result = []
        for obj in annotation.objects:
            if obj.name not in classes:
                continue
            x, y, width, height = self.get_object_params(obj, annotation.size)
            result.append("%d %.6f %.6f %.6f %.6f" % (classes[obj.name], x, y, width, height))
        return "\n".join(result)

    @staticmethod
    def get_object_params(obj, size):
        image_width = 1.0 * size.width
        image_height = 1.0 * size.height

        box = obj.box
        absolute_x = box.xmin + 0.5 * (box.xmax - box.xmin)
        absolute_y = box.ymin + 0.5 * (box.ymax - box.ymin)

        absolute_width = box.xmax - box.xmin
        absolute_height = box.ymax - box.ymin

        x = absolute_x / image_width
        y = absolute_y / image_height
        width = absolute_width / image_width
        height = absolute_height / image_height

        return x, y, width, height

    @staticmethod
    def darknet_filename_format(filename):
        pre, ext = os.path.splitext(filename)
        return "%s.txt" % pre

In [5]:
import os
import sys

xml_dir = os.path.join(os.path.dirname(os.path.realpath('__file__')), 'xml')
if not os.path.exists(xml_dir):
    print("Provide the correct folder for xml files.")
    sys.exit()

out_dir = os.path.join(os.path.dirname(os.path.realpath('__file__')), 'out')
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

if not os.access(out_dir, os.W_OK):
    print("%s folder is not writeable." % out_dir)
    sys.exit()
    
class_file = os.path.join(os.path.dirname(os.path.realpath('__file__')), 'classes.txt')

if not os.access(class_file, os.F_OK):
    print("%s file is missing." % class_file)
    sys.exit()

if not os.access(class_file, os.R_OK):
    print("%s file is not readable." % class_file)
    sys.exit()

img_dir = os.path.join(os.path.dirname(os.path.realpath('__file__')),'img')
    
transformer = Transformer(xml_dir=xml_dir, out_dir=out_dir, class_file=class_file, img_dir=img_dir)
transformer.transform()